In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('Train.csv')

In [3]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


# Plan
- Get used to the data
- Data clearning
- Exploratory data analysis
- Check whether the target is skewed.
- Handle missing values
- Handle outliers
- Check correlation between numerical variables
- Add features if needed
- Skewed data
- Scale data
- Get dummies and label encoding
- Train-Test-Val split
- Train model

# Get used to the data

In [5]:
for col in df.columns:
    print('###', col, ':\n')
    if len(df[col].unique()) >= 10:
        print(df[col].unique()[:10], ' (len = {})\n'.format(len(df[col].unique())))
    else:
        print(df[col].unique(), ' (len = {})\n'.format(len(df[col].unique())), '\n')
    print('Dtype: {}'.format(df[col].dtype))
    print('Number of null values: {}'.format(df[col].isnull().sum()), '\n')
    print('---------------------\n')

### ID :

[462809 462643 466315 461735 462669 461319 460156 464347 465015 465176]  (len = 8068)

Dtype: int64
Number of null values: 0 

---------------------

### Gender :

['Male' 'Female']  (len = 2)
 

Dtype: object
Number of null values: 0 

---------------------

### Ever_Married :

['No' 'Yes' nan]  (len = 3)
 

Dtype: object
Number of null values: 140 

---------------------

### Age :

[22 38 67 40 56 32 33 61 55 26]  (len = 67)

Dtype: int64
Number of null values: 0 

---------------------

### Graduated :

['No' 'Yes' nan]  (len = 3)
 

Dtype: object
Number of null values: 78 

---------------------

### Profession :

['Healthcare' 'Engineer' 'Lawyer' 'Entertainment' 'Artist' 'Executive'
 'Doctor' 'Homemaker' 'Marketing' nan]  (len = 10)

Dtype: object
Number of null values: 124 

---------------------

### Work_Experience :

[ 1. nan  0.  4.  9. 12.  3. 13.  5.  8.]  (len = 16)

Dtype: float64
Number of null values: 829 

---------------------

### Spending_Score :

['Low' 

In [6]:
# Comments

'''
- Drop ID
- Age => AgeGroup
- Check the relation between nans of Profession and Work_Experience 
''';